In [1]:
import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt  
import seaborn as seabornInstance 
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from scipy.stats.stats import pearsonr
import pickle
import statistics

In [2]:
cdk_desc = pd.read_csv(r'data_TA/cdk_desc.csv')
mordred_desc = pd.read_csv(r'data_TA/mordred_desc.csv')
padel_desc = pd.read_csv(r'data_TA/padel_desc.csv')
pIC50_target = pd.read_excel(r'data_TA/pIC50_target.xlsx')

# cdk_desc.shape, mordred_desc.shape, padel_desc.shape, pIC50_target.shape
# pIC50_target

In [3]:
#kolom yang dihapus
cdk_desc = cdk_desc.drop(["Title"], axis=1)
mordred_desc = mordred_desc.drop(["name"], axis=1)
padel_desc = padel_desc.drop(["Name"], axis=1)
pIC50_target = pIC50_target.drop(['No.'], axis=1)

cdk_desc.shape, mordred_desc.shape, padel_desc.shape

((134, 286), (134, 1613), (134, 1875))

In [4]:
dataset = pd.concat([cdk_desc,mordred_desc,padel_desc, pIC50_target], axis=1)
dataset

,ALogP,ALogp2,AMR,BCUTw-1l,BCUTw-1h,BCUTc-1l,BCUTc-1h,BCUTp-1l,BCUTp-1h,PPSA-1,...,P2s,E1s,E2s,E3s,Ts,As,Vs,Ks,Ds,pIC50
0,0.2908,0.084565,75.7828,11.994344,19.000412,-0.326879,0.204396,3.983285,9.371560,356.240964,...,0.230608,0.561977,0.489222,0.523529,13.444158,47.047030,106.927129,0.459836,1.574728,5.842
1,1.7599,3.097248,106.0192,11.850000,19.000417,-0.326851,0.233856,3.984015,9.866476,482.343937,...,0.124809,0.554310,0.365546,0.392117,25.097284,103.740040,238.275628,0.709610,1.311973,6.292
2,0.0863,0.007448,85.3487,11.988451,19.000414,-0.360020,0.278314,3.983783,9.619386,512.582748,...,0.093133,0.554587,0.403332,0.402626,21.435407,63.448720,134.077300,0.764904,1.360546,5.469
3,1.5280,2.334784,111.7861,11.996640,19.000419,-0.335855,0.233992,3.984031,9.937788,498.914595,...,0.123379,0.544054,0.403809,0.370641,28.756514,130.576181,305.286387,0.723444,1.318503,6.310
4,0.6093,0.371246,103.8623,11.850000,19.000417,-0.326858,0.235369,3.984004,9.824791,496.385973,...,0.141888,0.581195,0.397869,0.322742,24.574375,101.480573,223.519513,0.700411,1.301806,6.387
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,2.4573,6.038323,128.9326,11.850000,31.973074,-0.326855,0.202569,4.563459,14.040421,497.131970,...,0.152839,0.455611,0.377196,0.437523,29.375999,164.436509,430.436141,0.651397,1.270330,7.142
130,1.1899,1.415862,117.8384,11.850000,31.972073,-0.326866,0.204219,3.984556,12.441325,554.079784,...,0.364984,0.561717,0.357159,0.480373,24.895532,165.105971,384.973632,0.409651,1.399248,7.026
131,1.3954,1.947141,118.0548,11.850000,31.972073,-0.326871,0.204315,3.984551,12.439904,464.821377,...,0.368969,0.550420,0.363275,0.480499,24.939796,166.299510,388.900703,0.409199,1.394195,7.259
132,1.1735,1.377102,124.3016,11.996133,31.972073,-0.377504,0.204290,3.984572,12.458502,589.900158,...,0.336061,0.533630,0.431886,0.421620,22.937123,144.035418,372.377249,0.368102,1.387136,7.301


In [5]:
# drop kolom yang ada duplikat
dataset = dataset.loc[:,~dataset.columns.duplicated()]
dataset.shape

(134, 2981)

In [6]:
# drop kolom yang ada 0
dataset = dataset[(dataset.sum(axis=1) != 0)]
dataset.shape

(134, 2981)

In [7]:
# drop kolom yang ada nan
dataset.dropna(axis=1, inplace=True)
dataset.shape

(134, 2833)

In [8]:
dataset.isna().values.any()

False

In [9]:
train, test = train_test_split(dataset, test_size = 0.2, random_state = 10)

train.shape, test.shape
# test

((107, 2833), (27, 2833))

In [10]:
for i in train.columns:
    if (i != 'flag'):
        unique,counts = np.unique(train[i], return_counts=True)
        common_value = max(counts)
        similarity = common_value/len(train)*100
        temp_dev  = np.std(train[i].astype(float))
        if similarity >= 50:
#             print('Drop column: ',i)
            train.drop(i,axis=1,inplace=True)
train.shape

C:\Users\Bams\anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


(107, 2002)

In [11]:
#column standard deviation
train.std()

ALogP         0.852570
ALogp2        2.493731
AMR          15.741025
BCUTw-1l      0.063852
BCUTw-1h      6.017115
               ...    
As           49.501895
Vs          123.772430
Ks            0.162976
Ds            0.105847
pIC50         1.079135
Length: 2002, dtype: float64

In [12]:
deviation = 0.95
train = train.drop(train.std()[train.std() < deviation].index.values, axis=1)
train

,ALogp2,AMR,BCUTw-1h,BCUTp-1h,PPSA-1,PPSA-2,PPSA-3,PNSA-1,PNSA-2,PNSA-3,...,L2i,Ti,Ai,Vi,L1s,L2s,Ts,As,Vs,pIC50
14,0.147763,97.1397,19.000415,9.775956,580.687732,1323.815573,39.451850,245.989486,-560.791446,-55.425797,...,2.205607,23.788970,78.675719,170.995773,20.411635,2.136428,24.052696,77.534552,167.201273,6.374
87,3.198375,125.5069,19.000421,10.009326,520.840294,1703.142641,32.547639,255.796786,-836.452976,-52.385169,...,6.733735,22.297547,124.807233,281.810783,13.933197,6.656099,21.971273,121.194673,271.331551,5.678
35,0.169909,87.4056,19.000415,9.728512,458.279076,942.419124,25.183793,200.585946,-412.491081,-37.949694,...,6.117165,17.796741,87.167449,201.135527,10.334931,5.895541,17.744310,85.500311,195.482780,8.000
20,0.692890,110.7465,19.000419,9.876469,521.509584,1536.625077,41.443855,339.255543,-999.614563,-78.388971,...,3.566173,27.709233,115.253666,246.935877,23.809417,3.558004,28.572037,117.681218,248.301064,6.824
114,2.256004,124.7532,31.973075,13.338755,442.882752,1141.698191,31.363312,397.876762,-1025.678190,-70.786093,...,6.123936,24.159487,141.517368,355.940247,16.376656,6.173064,24.417220,143.205753,356.416312,8.108
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,1.190281,124.3204,31.973074,13.339288,481.262084,1135.961329,29.714406,368.782047,-870.465715,-55.124090,...,7.214347,22.923451,133.828958,304.632319,14.196442,7.280135,22.826892,132.352162,294.736866,7.387
64,1.568005,142.3687,31.974077,12.806806,599.393756,1903.782605,38.026163,427.712901,-1358.493264,-74.035947,...,12.123501,29.596260,233.273145,522.823124,16.034681,12.277262,29.606310,233.508021,517.925939,6.821
15,1.886777,113.2754,19.000419,9.918155,549.241338,1548.450567,46.853070,346.645068,-977.280322,-76.279602,...,3.658802,28.115392,118.556366,253.049750,24.056900,3.646627,28.885611,120.474440,253.060199,6.652
125,8.747398,129.0211,34.968855,13.897981,567.298297,1480.568473,41.986979,444.610971,-1160.371873,-73.309173,...,5.356493,31.164207,173.072700,390.824207,24.516772,5.457567,31.318280,174.085678,385.225921,5.958


In [13]:
# -----------REMOVE FEATURE WITH LOW CORRELATION TO Y RESPONSE---------------------
label_ = train.columns.values
pic50 = train.iloc[:,-1]
corr_y = [pearsonr(train.iloc[:,i],pic50) for i in range(train.shape[1])]
corr_y = [np.abs(corr_y[i][0]) for i in range(train.shape[1])]
corr_lim = 0.20
hi_corr = []
for i in range(len(corr_y)):
    if corr_y[i] > corr_lim:
        hi_corr.append(i)
label_idx = label_[hi_corr]
train = train.loc[:,label_idx]
print("descriptor number after removing low correlation with target: {}".format(train.shape[1]))

descriptor number after removing low correlation with target: 184


In [14]:
# -----------REMOVE FEATURE WITH HIGH CORRELATION TO OTHER FEATURE---------------------
# re-calculate correlation with pic50
label_ = train.columns.values
corr_y = [pearsonr(train.iloc[:,i],pic50) for i in range(train.shape[1])]
corr_y = [np.abs(corr_y[i][0]) for i in range(train.shape[1])]
desc_num = train.shape[1]
# calculate correlation for each descriptor
corr_matrix = np.corrcoef(train.T)
corr_lim = 0.80
low_corr = np.arange(desc_num).tolist()
tmp = np.arange(desc_num).tolist()
for i in np.arange(desc_num):
    tmp.remove(i)
    for j in tmp:
        corr_ = np.abs(corr_matrix[i,j])
        if corr_ >= corr_lim:
            if corr_y[i] > corr_y[j]:
                if j in low_corr:
                    low_corr.remove(j)
            else:
                if i in low_corr:
                    low_corr.remove(i)
label_idx = label_[low_corr]
train = train.loc[:,label_idx]
print("descriptor number after removing high correlation descriptor: {}".format(train.shape[1]))

descriptor number after removing high correlation descriptor: 67


In [15]:
#print datatest dan datatrain
test.to_csv('data_TA/new_test.csv', encoding='utf-8')
train.to_csv('data_TA/new_train.csv', encoding='utf-8')

In [16]:
label_ = train.columns.values
pickle.dump(label_, open('data_TA/train_Label.p','wb'))

In [17]:
# dataset.describe()